# Notebook pro trénink s destilací nad datasetem CIFAR10
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR10, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

In [1]:
%pip install transformers[torch] huggingface_hub datasets evaluate torchvision optuna

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import knihoven a definice metod

In [2]:
from transformers import Trainer, TrainingArguments, MobileNetV2Config, MobileNetV2ForImageClassification, EarlyStoppingCallback
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
import numpy as np
import evaluate
import random
import pickle
import optuna
import torch
import os 

Resetování náhodného seedu pro replikovatelnost výsledků.
Zřejmě je možné části odebrat.

TODO: Odebrat zbytečná nastavení.

In [3]:
def reset_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

Nový wrapper, který pracuje přímo se soubory staženého a upraveného datasetu CIFAR10.
Využití načtení pomocí metody jako dříve není možné kvůli jiné checksum. 

Zároveň se již dotahují logity přímo z datasetu.

In [4]:
class CustomCIFAR10(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None):
        self.root = root
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        self.data = []
        self.targets = []
        self.logits = []
        
        if self.train:
             for i in range(1, 6):
                 data_file = os.path.join(self.root, 'cifar-10-batches-py', f'data_batch_{i}')
                 with open(data_file, 'rb') as fo:
                     dict = pickle.load(fo, encoding='bytes')
                     self.data.append(dict[b'data'])
                     self.targets.extend(dict[b'labels'])
                     self.logits.extend(dict[b'logits'])  
        else:
            data_file = os.path.join(self.root, 'cifar-10-batches-py', 'test_batch')
            with open(data_file, 'rb') as fo:
                dict = pickle.load(fo, encoding='bytes')
                self.data.append(dict[b'data'])
                self.targets.extend(dict[b'labels'])
                self.logits.extend(dict[b'logits'])  

        self.data = np.concatenate(self.data, axis=0)
        self.targets = np.array(self.targets)
        self.logits = np.array(self.logits)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index].reshape(3, 32, 32).transpose(1, 2, 0)
        label = self.targets[index]
        logit = self.logits[index]
        
        image = Image.fromarray(image.astype('uint8'), 'RGB')
        logit = torch.tensor(logit, dtype=torch.float)
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)
            
        return {
            'pixel_values': image,
            'labels': label,
            'logits': logit
        }


Definice accuracy metriky pro trénování modelu.

In [5]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    accuracy = accuracy_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels)
    precision = precision_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')
    recall = recall_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')
    f1 = f1_metric.compute(predictions=np.argmax(predictions, axis=1), references=labels, average='macro')

    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

Trénovací argumenty pro trainer. 

In [6]:
class Custom_training_args(TrainingArguments):
    def __init__(self, lambda_param, temperature, *args, **kwargs):
        super().__init__(*args, **kwargs)    
        self.lambda_param = lambda_param
        self.temperature = temperature

In [7]:
def get_training_args(output_dir:str, logging_dir:str, remove_unused_columns:bool):
    return (
        Custom_training_args(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5, #Defaultní hodnota 
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=20,
        weight_decay=0.01,
        seed = 42,  #Defaultní hodnota 
        metric_for_best_model="f1",
        load_best_model_at_end=True,
        fp16=False, 
        logging_dir=logging_dir,
        remove_unused_columns=remove_unused_columns,
        lambda_param = 0.5, 
        temperature = 5
    ))

Náhodně inicializovaný MobileNetV2.

In [8]:
def get_random_init_mobilenet():
    reset_seed(42)
    student_config = MobileNetV2Config()
    student_config.num_labels = 10
    return MobileNetV2ForImageClassification(student_config)

In [9]:
reset_seed(42)

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test = CustomCIFAR10(root='../data/10-logits', train=False, transform=transform)
train = CustomCIFAR10(root='../data/10-logits', train=True, transform=transform)

### Standardní trénink náhodně inicializovaného modelu. 

In [12]:
training_args = get_training_args("./results/cifar10-random", './logs/cifar10-random', True)

In [13]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3)
    }

In [14]:
pruner = optuna.pruners.HyperbandPruner(min_resource=5, reduction_factor=4)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [15]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [16]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    n_trials=80
)

[I 2025-01-01 23:59:01,142] A new study created in memory with name: no-name-7c8c698d-acf7-4823-8e1e-c939c40ae55e


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.870100,1.732741,0.367800,0.372398,0.367800,0.344483
2,1.686700,1.530936,0.436400,0.462839,0.436400,0.436615
3,1.518200,1.405519,0.490700,0.496632,0.490700,0.483040
4,1.398600,1.321123,0.524700,0.526839,0.524700,0.514257
5,1.309000,1.367839,0.518300,0.582725,0.518300,0.525557
6,1.255500,1.170599,0.587500,0.578888,0.587500,0.576408
7,1.187700,1.262417,0.564600,0.591695,0.564600,0.553731
8,1.126400,1.838385,0.483700,0.560011,0.483700,0.464588
9,1.055700,1.095330,0.614900,0.646942,0.614900,0.607511
10,1.002800,0.970768,0.657500,0.669641,0.657500,0.659654


[I 2025-01-02 00:47:58,443] Trial 0 finished with value: 0.6166393892685925 and parameters: {'learning_rate': 1.0253509690168497e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.001}. Best is trial 0 with value: 0.6166393892685925.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.368300,2.179152,0.199900,0.191525,0.199900,0.160997
2,2.101700,1.920544,0.272100,0.279815,0.272100,0.243458
3,1.980100,1.825716,0.304000,0.306483,0.304000,0.292905
4,1.915600,1.773887,0.331800,0.329674,0.331800,0.317815
5,1.871700,1.732130,0.345000,0.362109,0.345000,0.336211
6,1.822400,1.681978,0.366800,0.372624,0.366800,0.357031
7,1.783100,1.726380,0.351800,0.357473,0.351800,0.341093
8,1.740400,2.282531,0.276800,0.327437,0.276800,0.247815
9,1.720700,1.607467,0.402300,0.418300,0.402300,0.383987
10,1.696300,1.560524,0.417400,0.425994,0.417400,0.413825


[I 2025-01-02 01:49:21,289] Trial 1 finished with value: 0.44523251373587086 and parameters: {'learning_rate': 2.6364803038431666e-06, 'per_device_train_batch_size': 32, 'weight_decay': 0.007}. Best is trial 0 with value: 0.6166393892685925.


...

[I 2025-01-02 10:25:49,006] Trial 77 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.446200,2.238760,0.177500,0.156460,0.177500,0.125903


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-01-02 10:28:52,219] Trial 78 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.398200,1.267022,0.539300,0.552113,0.539300,0.525163


[I 2025-01-02 10:31:58,127] Trial 79 pruned. 


## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

In [17]:
class ImageDistilTrainer(Trainer):
    def __init__(self, model_init, *args, **kwargs):
        self.model_init = model_init
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
       
        super().__init__(model_init=model_init, *args, **kwargs)
        
        self.student = self.model_init()
        self.temperature = self.args.temperature
        self.lambda_param = self.args.lambda_param

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.student.to(device)


    def compute_loss(self, model, inputs, return_outputs=False):
        logits = inputs.pop("logits")

        student_output = model(**inputs)
        self.lambda_param = self.args.lambda_param
        self.temperature = self.args.temperature
        
        soft_teacher = F.softmax(logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        student_target_loss = student_output.loss

        loss = ((1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss)
        return (loss, student_output) if return_outputs else loss

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [18]:
reset_seed(42)

In [19]:
training_args = get_training_args("./results/cifar10-random-KD", './logs/cifar10-random-KD', False)

In [20]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }

In [21]:
trainer = ImageDistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics,
    model_init=get_random_init_mobilenet,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [22]:
pruner = optuna.pruners.HyperbandPruner(min_resource=1, reduction_factor=4)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [23]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    n_trials=80
)

[I 2025-01-02 10:31:58,319] A new study created in memory with name: no-name-c949043e-bbe5-4784-8e94-3711ffa24f5e


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.826100,1.794448,0.356200,0.363023,0.356200,0.332477
2,1.663900,1.638281,0.429000,0.451496,0.429000,0.430032
3,1.506000,1.536666,0.490100,0.496502,0.490100,0.486227
4,1.396100,1.514397,0.516500,0.521185,0.516500,0.506952
5,1.311700,1.508902,0.527800,0.575985,0.527800,0.532914
6,1.262200,1.367924,0.586400,0.578568,0.586400,0.576121
7,1.198600,1.423449,0.576800,0.601245,0.576800,0.572559
8,1.125700,1.765431,0.509900,0.563286,0.509900,0.494695
9,1.058500,1.310685,0.626400,0.646086,0.626400,0.620627
10,1.029100,1.226045,0.662000,0.674270,0.662000,0.660745


[I 2025-01-02 11:14:36,692] Trial 0 finished with value: 0.6493830756403727 and parameters: {'learning_rate': 1.0253509690168497e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.001, 'lambda_param': 0.1, 'temperature': 2.0}. Best is trial 0 with value: 0.6493830756403727.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.650800,1.645658,0.641600,0.651808,0.641600,0.630255
2,0.467800,1.807167,0.725200,0.751262,0.725200,0.725268
3,0.372000,1.774964,0.796000,0.797620,0.796000,0.793375
4,0.311600,1.782879,0.820700,0.821734,0.820700,0.819950
5,0.266400,1.785201,0.801300,0.828130,0.801300,0.804786
6,0.232700,1.714340,0.823700,0.833181,0.823700,0.820774
7,0.202900,1.730780,0.816000,0.830533,0.816000,0.814600
8,0.177800,1.671794,0.767400,0.791711,0.767400,0.761982
9,0.154400,1.754976,0.850000,0.855467,0.850000,0.849485
10,0.136000,1.758484,0.853400,0.860455,0.853400,0.854400


[I 2025-01-02 12:06:48,211] Trial 1 finished with value: 0.8775070142695549 and parameters: {'learning_rate': 0.00021766241123453658, 'per_device_train_batch_size': 32, 'weight_decay': 0.01, 'lambda_param': 0.9, 'temperature': 3.0}. Best is trial 1 with value: 0.8775070142695549.


...

[I 2025-01-02 19:20:27,459] Trial 77 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.594300,1.438909,0.170700,0.148205,0.170700,0.118925


[I 2025-01-02 19:23:30,635] Trial 78 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.609300,1.698447,0.615400,0.631447,0.615400,0.606751


[I 2025-01-02 19:26:47,257] Trial 79 pruned. 
